# 第4章: 言語解析

問題30から問題35までは、以下の文章`text`（太宰治の『走れメロス』の冒頭部分）に対して、言語解析を実施せよ。問題36から問題39までは、国家を説明した文書群（日本語版ウィキペディア記事から抽出したテキスト群）をコーパスとして、言語解析を実施せよ。

In [2]:
text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

In [2]:
# 事前準備
# pip install mecab-python3
# pip install pandas
# pip install unidic
# python -m unidic download
# pip install japanize-matplotlib

## 30. 動詞
文章`text`に含まれる動詞をすべて表示せよ。

In [6]:
import MeCab
mecab = MeCab.Tagger()

for i, line in enumerate(text.splitlines()):
    for parsed_line in mecab.parse(line).splitlines():
        if parsed_line.split("\t")[0] != "EOS":
            if parsed_line.split("\t")[1].split(",")[0] == "動詞":
                print("文中の動詞-> ", parsed_line.split("\t")[0])
                print(parsed_line.split("\t")[1])
                print("")

文中の動詞->  し
動詞,非自立可能,,,サ行変格,連用形-一般,スル,為る,し,シ,する,スル,和,"","","","","","",用,シ,スル,シ,スル,"0","C5","",5370298291593857,19537

文中の動詞->  除か
動詞,一般,,,五段-カ行,未然形-一般,ノゾク,除く,除か,ノゾカ,除く,ノゾク,和,"","","","","","",用,ノゾカ,ノゾク,ノゾカ,ノゾク,"0","C2","",8004178429485633,29119

文中の動詞->  なら
動詞,非自立可能,,,五段-ラ行,未然形-一般,ナル,成る,なら,ナラ,なる,ナル,和,"","","","","","",用,ナラ,ナル,ナラ,ナル,"1","C1","",7713357570384449,28061

文中の動詞->  し
動詞,非自立可能,,,サ行変格,連用形-一般,スル,為る,し,シ,する,スル,和,"","","","","","",用,シ,スル,シ,スル,"0","C5","",5370298291593857,19537

文中の動詞->  わから
動詞,一般,,,五段-ラ行,未然形-一般,ワカル,分かる,わから,ワカラ,わかる,ワカル,和,"","","","","","",用,ワカラ,ワカル,ワカラ,ワカル,"2","C1","",11319480831582785,41180

文中の動詞->  ある
動詞,非自立可能,,,五段-ラ行,終止形-一般,アル,有る,ある,アル,ある,アル,和,"","","","","","",用,アル,アル,アル,アル,"1","C3","",334260158472875,1216

文中の動詞->  吹き
動詞,一般,,,五段-カ行,連用形-一般,フク,吹く,吹き,フキ,吹く,フク,和,"","","","","","",用,フキ,フク,フキ,フク,"1,2","C1","",8958554488840833,32591

文中の動詞->  遊ん
動詞,一般,,,五段-バ行,連用形-撥音便,アソブ,遊ぶ,遊ん,アソン,遊ぶ,アソブ,和,"","","","","","",用,アソン,アソブ,アソン,アソブ,"0","C2","",198470505996933,722

文中の動詞-> 

## 31. 動詞の原型
文章`text`に含まれる動詞と、その原型をすべて表示せよ。

In [7]:
import MeCab
mecab = MeCab.Tagger()

for i, line in enumerate(text.splitlines()):
    for parsed_line in mecab.parse(line).splitlines():
        if parsed_line.split("\t")[0] != "EOS":
            if parsed_line.split("\t")[1].split(",")[0] == "動詞":
                print("文中の動詞-> ", parsed_line.split("\t")[0])
                print(parsed_line.split("\t")[1])
                print("動詞の原形-> ", parsed_line.split("\t")[1].split(",")[10])
                print("")

文中の動詞->  し
動詞,非自立可能,,,サ行変格,連用形-一般,スル,為る,し,シ,する,スル,和,"","","","","","",用,シ,スル,シ,スル,"0","C5","",5370298291593857,19537
動詞の原形->  する

文中の動詞->  除か
動詞,一般,,,五段-カ行,未然形-一般,ノゾク,除く,除か,ノゾカ,除く,ノゾク,和,"","","","","","",用,ノゾカ,ノゾク,ノゾカ,ノゾク,"0","C2","",8004178429485633,29119
動詞の原形->  除く

文中の動詞->  なら
動詞,非自立可能,,,五段-ラ行,未然形-一般,ナル,成る,なら,ナラ,なる,ナル,和,"","","","","","",用,ナラ,ナル,ナラ,ナル,"1","C1","",7713357570384449,28061
動詞の原形->  なる

文中の動詞->  し
動詞,非自立可能,,,サ行変格,連用形-一般,スル,為る,し,シ,する,スル,和,"","","","","","",用,シ,スル,シ,スル,"0","C5","",5370298291593857,19537
動詞の原形->  する

文中の動詞->  わから
動詞,一般,,,五段-ラ行,未然形-一般,ワカル,分かる,わから,ワカラ,わかる,ワカル,和,"","","","","","",用,ワカラ,ワカル,ワカラ,ワカル,"2","C1","",11319480831582785,41180
動詞の原形->  わかる

文中の動詞->  ある
動詞,非自立可能,,,五段-ラ行,終止形-一般,アル,有る,ある,アル,ある,アル,和,"","","","","","",用,アル,アル,アル,アル,"1","C3","",334260158472875,1216
動詞の原形->  ある

文中の動詞->  吹き
動詞,一般,,,五段-カ行,連用形-一般,フク,吹く,吹き,フキ,吹く,フク,和,"","","","","","",用,フキ,フク,フキ,フク,"1,2","C1","",8958554488840833,32591
動詞の原形->  吹く

文中の動詞->  遊ん
動詞,一般,,,五段-バ行,連用形-撥音便,アソブ,遊ぶ,遊ん,アソン,遊

## 32. 「AのB」
文章`text`において、2つの名詞が「の」で連結されている名詞句をすべて抽出せよ。

In [3]:
import MeCab
mecab = MeCab.Tagger()

for i, line in enumerate(text.splitlines()):
    parsed_list = mecab.parse(line).splitlines()
    for index, parsed_line in enumerate(parsed_list):
        if parsed_line.split("\t")[0] != "EOS":
            if parsed_line.split("\t")[1].split(",")[0] == "助詞":
                if parsed_line.split("\t")[0] == "の":
                    noun1 = parsed_list[index-1].split("\t")[0]
                    particle = parsed_line.split("\t")[1].split(",")[10]
                    noun2 = parsed_list[index+1].split("\t")[0]
                    print(f"名詞句-> {noun1}{particle}{noun2}")

名詞句-> 暴虐の王
名詞句-> 村の牧人


## 33. 係り受け解析

文章`text`に係り受け解析を適用し、係り元と係り先のトークン（形態素や文節などの単位）をタブ区切り形式ですべて抽出せよ。

## 34. 主述の関係
文章`text`において、「メロス」が主語であるときの述語を抽出せよ。

## 35. 係り受け木
「メロスは激怒した。」の係り受け木を可視化せよ。

## 36. 単語の出現頻度

問題36から39までは、Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](/data/jawiki-country.json.gz)をコーパスと見なし、統計的な分析を行う。

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が"title"キーに、記事本文が"text"キーの辞書オブジェクトに格納され、そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される

まず、第3章の処理内容を参考に、Wikipedia記事からマークアップを除去し、各記事のテキストを抽出せよ。そして、コーパスにおける単語（形態素）の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

## 37. 名詞の出現頻度
コーパスにおける名詞の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

## 38. TF・IDF
日本に関する記事における名詞のTF・IDFスコアを求め、TF・IDFスコア上位20語とそのTF, IDF, TF・IDFを表示せよ。

## 39. Zipfの法則
コーパスにおける単語の出現頻度順位を横軸、その出現頻度を縦軸として、両対数グラフをプロットせよ。